In [156]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GROQ_API_KEY")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [157]:
!pip install groq chromadb sentence-transformers langchain python-dotenv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [158]:
import os
import groq
from chromadb import Client, Settings
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
import json
from typing import List, Dict, Set, Tuple
from datetime import datetime
import uuid
from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from langchain.chains import LLMChain
import re

load_dotenv()

class MultiAgentDungeonMaster:
    def __init__(self, persist_directory: str = "./chroma_multiplayer_db"):
        # Initialize Groq client
        self.client = groq.Groq(api_key=secret_value_0)
        
        # Initialize embedding model
        self.embedder = SentenceTransformer('BAAI/bge-small-en-v1.5')
        
        # Initialize ChromaDB
        self.persist_directory = persist_directory
        os.makedirs(self.persist_directory, exist_ok=True)
        
        try:
            self.chroma_client = Client(settings=Settings(
                persist_directory=self.persist_directory,
                is_persistent=True
            ))
        except Exception as e:
            print(f"Warning: Could not initialize ChromaDB with persistence: {e}")
            self.chroma_client = Client()
        
        # Collections for different memory types
        self.collections = {
            "global_events": self._get_or_create_collection("global_events"),
            "player_memories": self._get_or_create_collection("player_memories"), 
            "npc_memories": self._get_or_create_collection("npc_memories"),
            "quests": self._get_or_create_collection("quests")
        }
        
        # Initialize specialized agents
        self.dm_agent = DungeonMasterAgent(self.client)
        self.memory_agent = MemoryAgent(self.client)
        self.lore_keeper = LoreKeeperAgent(self.client)
        
        # Game state
        self.players = {}
        self.active_players = set()
        self.game_session_id = str(uuid.uuid4())[:8]
        self.turn_count = 0
        self.player_working_memory = {}
        self.global_working_memory = []
        
        print(f"🎮 Multi-Agent Dungeon Master initialized! Session ID: {self.game_session_id}")
    
    def _get_or_create_collection(self, name: str):
        """Safely get or create a ChromaDB collection"""
        try:
            return self.chroma_client.get_or_create_collection(name)
        except Exception as e:
            return self.chroma_client.create_collection(name)

In [159]:
class DungeonMasterAgent:
    def __init__(self, client):
        self.client = client
        self.prompt_template = PromptTemplate(
            input_variables=["party_context", "player_context", "memory_context", "recent_context", "player_input", "player_character"],
            template="""You are the Dungeon Master in a fantasy RPG. Your role is to create immersive, descriptive storytelling.

PARTY CONTEXT:
{party_context}

PLAYER CONTEXT:
{player_context}

RELEVANT PAST EVENTS:
{memory_context}

RECENT CONVERSATION:
{recent_context}

CURRENT ACTION:
{player_character}: {player_input}

As the Dungeon Master, you must:
1. Write vivid, atmospheric descriptions
2. Advance the story based on player actions
3. Create challenges and opportunities
4. Maintain dramatic pacing
5. React to player choices meaningfully
6. **IMPORTANT: Keep responses under 400 words to avoid cutoff**

Respond with rich sensory details and narrative depth. Describe what the player sees, hears, feels, and experiences.

DM Response:"""
        )

    def generate_response(self, context: Dict , temperature: float = 0.7) -> str:
        try:
            prompt = self.prompt_template.format(
                party_context=context.get("party_context", ""),
                player_context=context.get("player_context", ""),
                memory_context=context.get("memory_context", ""),
                recent_context=context.get("recent_context", ""),
                player_input=context.get("player_input", ""),
                player_character=context.get("player_character", "Player")
                )
            
            response = self.client.chat.completions.create(
                model="llama-3.1-8b-instant",
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature,
                max_tokens=400
                )
            
            return response.choices[0].message.content.strip()
        except Exception as e:
            return f"The dungeon grows silent... (Error: {str(e)})"

class MemoryAgent:
    def __init__(self, client):
        self.client = client
        # Use a simple string template instead of PromptTemplate if there are issues
        self.template = """Analyze the current game action and extract key factual information for long-term memory.

CURRENT ACTION:
{current_action}

DUNGEON MASTER RESPONSE:
{dm_response}

RECENT HISTORY (last 3 turns):
{recent_history}

Extract 1-3 key facts that should be remembered for future reference. Focus on:
- Important events that occurred
- Character introductions or developments
- Locations discovered or described
- Items found or important objects
- Player achievements or failures
- NPC interactions or relationships
- Quest progress or new objectives

**IMPORTANT: Keep each fact concise (1 sentence max)**

Format as concise factual statements. Each fact should be standalone and meaningful.

Key Facts:"""
    
    def extract_memories(self, context: Dict) -> List[str]:
        try:
            prompt = self.template.format(
                current_action=context.get("current_action", ""),
                dm_response=context.get("dm_response", ""),
                recent_history=context.get("recent_history", "")
                )
            
            response = self.client.chat.completions.create(
                model="llama-3.1-8b-instant", 
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
                max_tokens=200
                )
            
            text = response.choices[0].message.content.strip()
            facts = []
            for line in text.split('\n'):
                line = line.strip()
                if line and not line.startswith('Key Facts:') and len(line) > 10:
                    clean_line = re.sub(r'^[\d\-•\.\s]+', '', line)
                    if clean_line:
                        facts.append(clean_line)
            
            return facts if facts else [text[:150]]
        except Exception as e:
            print(f"Memory agent error: {e}")
            return []


class LoreKeeperAgent:
    def __init__(self, client):
        self.client = client
        self.consistency_template = PromptTemplate(
                input_variables=["dm_response", "retrieved_memories", "current_context"],
                template="""You are the Lore Keeper, ensuring story consistency and continuity.

RETRIEVED PAST MEMORIES:
{retrieved_memories}

CURRENT GAME CONTEXT:
{current_context}

PROPOSED DM RESPONSE:
{dm_response}

Your task:
1. Check for contradictions with established facts
2. Identify missing continuity elements
3. Suggest minor adjustments to maintain consistency
4. Flag major contradictions that need rewriting
5. **IMPORTANT: Keep response concise and focused**

If the response is consistent, return it as is.
If minor adjustments are needed, provide the improved version.
If major contradictions exist, explain the issues briefly.

Focus only on factual consistency, not writing style.

Evaluated Response:"""
                )

    def check_consistency(self, context: Dict) -> Tuple[str, List[str]]:
        try:
            prompt = self.consistency_template.format(
                dm_response=context.get("dm_response", ""),
                retrieved_memories=context.get("retrieved_memories", ""),
                current_context=context.get("current_context", "")
                )
            
            response = self.client.chat.completions.create(
                model="llama-3.1-8b-instant",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.2,
                max_tokens=500
                )
            
            result = response.choices[0].message.content.strip()
            
            # Parse the response to extract final text and notes
            if "Final Response:" in result:
                parts = result.split("Final Response:")
                notes = parts[0].strip()
                final_response = parts[1].strip()
            elif "Improved:" in result:
                parts = result.split("Improved:")
                notes = parts[0].strip()
                final_response = parts[1].strip()
            else:
                # Assume the whole response is the final text
                final_response = result
                notes = ["Response appears consistent"]
            
            return final_response, [notes] if isinstance(notes, str) else notes
            
        except Exception as e:
            print(f"Lore keeper error: {e}")
            return context.get("dm_response", ""), [f"Consistency check failed: {str(e)}"]

In [160]:
class EnhancedMultiplayerDungeonMaster(MultiAgentDungeonMaster):
    def __init__(self, persist_directory: str = "./chroma_multiplayer_db"):
        super().__init__(persist_directory)
        self.current_npc = None
        self.shared_world_state = {
            "location": "Crossroads Inn",  # Default starting location
            "time": "evening", 
            "weather": "stormy"
        }
    
    def add_player(self, player_name: str, character_name: str = None) -> str:
        player_id = str(uuid.uuid4())[:8]
        character = character_name or player_name
    
        self.players[player_id] = {
            "name": player_name,
            "character": character,
            "joined_at": datetime.now(),
            "last_action": None
        }
    
        self.active_players.add(player_id)
        self.player_working_memory[player_id] = []
    
    # FIX: Remove the old metadata dict - use named parameters instead
        intro_memory = f"Player {player_name} joined the game as {character}"
        self.add_to_memory(
            content=intro_memory, 
            memory_type="player_join", 
            player_id=player_id,  # Pass as named parameter
            importance=0.7  # Medium importance for player joins
        )
    
        print(f"🎮 Player {player_name} joined as {character} (ID: {player_id})")
        return player_id
    def _calculate_importance(self, content: str, memory_type: str) -> float:
        """Automatically assign importance score based on content"""
        base_score = 1.0
    
    # Boost importance for critical events
        important_keywords = ['dragon', 'king', 'queen', 'treasure', 'artifact', 
                         'died', 'defeated', 'victory', 'curse', 'prophecy']
        minor_keywords = ['looked', 'walked', 'noticed', 'saw', 'heard']
    
        content_lower = content.lower()
    
        if any(keyword in content_lower for keyword in important_keywords):
            base_score = 0.9
        elif any(keyword in content_lower for keyword in minor_keywords):
            base_score = 0.3
        elif memory_type == "quest":
            base_score = 0.8
    
        return base_score
    
    def add_to_memory(self, content: str, memory_type: str = "event", 
                 importance: float = None, npc: str = None, 
                 location: str = None, player_id: str = None):
        """Store memories with rich metadata for contextual retrieval"""
        if importance is None:
            importance = self._calculate_importance(content, memory_type)
    
        embedding = self.embedder.encode(content).tolist()
    
    # FIX: Flatten metadata - no nested dicts!
        metadata = {
        "turn": self.turn_count,
        "timestamp": datetime.now().timestamp(),
        "importance": float(importance),  # Ensure it's float
        "type": memory_type,
        "npc": npc or "global",
        "location": location or "unknown",
        "player_id": player_id or "unknown",  # Store as string, not dict
        "session": self.game_session_id
        }
    
    # Remove None values to avoid metadata issues
        metadata = {k: v for k, v in metadata.items() if v is not None}
    
        memory_id = f"{memory_type}_{self.turn_count}_{str(uuid.uuid4())[:4]}"
    
    # Store in appropriate collection
        collection = self._get_collection_by_type(memory_type, npc)
        collection.add(
        embeddings=[embedding],
        documents=[content],
        metadatas=[metadata],  # Now it's a flat dict, not nested
        ids=[memory_id]
    )
    
        print(f"💾 [{memory_type.upper()}] {content} (importance: {importance:.1f})")
    

    def detect_context_type(self, player_input: str) -> str:
        """Detect the context type based on player input"""
        text_lower = player_input.lower()
    
    # Lore checks - memory recall, factual questions
        lore_keywords = ["remember", "recall", "who was", "what was", "when did", 
                    "where is", "tell me about", "know about", "history"]
        if any(keyword in text_lower for keyword in lore_keywords):
            return "lore_check"
        
    # Battle/action scenes - high intensity
        action_keywords = ["attack", "fight", "battle", "defend", "strike", "kill",
                      "charge", "shoot", "cast spell", "magic missile", "fireball",
                      "scream", "yell", "run", "flee", "danger", "blood", "die"]
        if any(keyword in text_lower for keyword in action_keywords):
            return "battle"
    
    # Exploration/observation - calm and detailed
        explore_keywords = ["look", "observe", "examine", "study", "search", "inspect",
                       "listen", "watch", "survey", "scan", "check", "peek"]
        if any(keyword in text_lower for keyword in explore_keywords):
            return "exploration"
    
    # Emotional/reflective moments
        emotion_keywords = ["afraid", "scared", "fear", "terrified", "panic", "nervous",
                       "sad", "cry", "mourn", "regret", "happy", "joy", "excited"]
        if any(keyword in text_lower for keyword in emotion_keywords):
            return "emotional"
    
    # Default storytelling mode
        return "story"

    def get_dynamic_temperature(self, player_input: str) -> float:
        """Get temperature based on detected context"""
        context = self.detect_context_type(player_input)
    
        temperature_map = {
            "lore_check": 0.3,      # Consistent, factual recall
            "exploration": 0.4,     # Detailed, focused observation
            "emotional": 0.6,       # Empathetic, reflective
            "story": 0.7,           # Balanced creativity
            "battle": 1.0,          # High drama, intense action
        }
    
        temp = temperature_map.get(context, 0.7)
    
    # Optional: Print emotion mode for debugging
        print(f"🎭 Emotion Mode: {context} | Temperature: {temp}")
    
        return temp

    
    def search_memories(self, query: str, npc_filter: str = None, 
                   location_filter: str = None, n_results: int = 5) -> List[Tuple[str, float]]:
        """Enhanced retrieval with recency + importance weighting"""
    
        query_embedding = self.embedder.encode(query).tolist()
        now = datetime.now().timestamp()
    
    # Get collections to search
        collections_to_search = self._get_relevant_collections(npc_filter)
    
        all_results = []
    
        for collection in collections_to_search:
            results = collection.query(
                query_embeddings=[query_embedding],
                n_results=n_results * 2,  # Get more for filtering
                include=["metadatas", "distances"]
            )
        
            if results and results['documents']:
                for i, (text, metadata, distance) in enumerate(zip(
                    results['documents'][0], 
                    results['metadatas'][0], 
                    results['distances'][0]
                )):
                # Calculate weighted score
                    similarity_score = 1 - distance
                    recency_weight = 1 / (1 + (now - metadata["timestamp"]) / 3600)  # Hour decay
                    importance_weight = metadata.get("importance", 1.0)
                
                # Contextual scoring formula
                    final_score = (0.6 * similarity_score + 
                             0.3 * recency_weight + 
                             0.1 * importance_weight)
                
                    all_results.append((text, final_score, metadata))
    
    # Sort by weighted score and return top results
        all_results.sort(key=lambda x: x[1], reverse=True)
        return [(text, score) for text, score, _ in all_results[:n_results]]
    
    def generate_dm_response(self, player_id: str, player_input: str) -> Tuple[str, Dict]:
        """Generate DM response using the multi-agent pipeline"""
        if player_id not in self.players:
            return "I don't recognize you. Please join the game first.", {}
    
        self.turn_count += 1
        player = self.players[player_id]

        
        print(f"\n🔄 Turn {self.turn_count} - Multi-agent processing for {player['character']}...")

        temperature = self.get_dynamic_temperature(player_input)
    # FIX: Define current_npc and current_location
        current_npc = self.current_npc
        current_location = self.shared_world_state.get("location", "unknown")
    
    # Step 1: Gather context
        context = self.build_enhanced_context(player_input, current_npc, current_location)
    
    # Step 2: Dungeon Master creates initial response
        print("   🤖 DM Agent: Creating narrative...")
        dm_response = self.dm_agent.generate_response(context,temperature)
    
    # Step 3: Memory Agent extracts key facts
        print("   🧠 Memory Agent: Extracting memories...")
        memory_context = {
            "current_action": f"{player['character']}: {player_input}",
            "dm_response": dm_response,
            "recent_history": "\n".join(self.player_working_memory.get(player_id, [])[-3:])
        }
        new_memories = self.memory_agent.extract_memories(memory_context)
    
    # Step 4: Save new memories - FIX THIS CALL
        for memory in new_memories:
            self.add_to_memory(
                content=memory, 
                memory_type="event", 
                player_id=player_id,  # Pass as named parameter
                npc=player['character']  # Character name as NPC
            )
    
    # Step 5: Lore Keeper checks consistency
        print("   📚 Lore Keeper: Checking consistency...")
        lore_context = {
            "dm_response": dm_response,
            "retrieved_memories": "\n".join(context.get("memory_context", "").split('\n')[:5]),
            "current_context": f"Player: {player['character']}, Action: {player_input}"
        }
        final_response, consistency_notes = self.lore_keeper.check_consistency(lore_context)
    
    # Step 6: Update working memory
        self._update_working_memory(player_id, player_input, final_response)
    
        agent_log = {
            "dm_initial": dm_response,
            "memories_extracted": new_memories,
            "consistency_notes": consistency_notes,
            "final_response": final_response
        }
    
        return final_response, agent_log

        
 
    def _get_collection_by_type(self, memory_type: str, npc: str = None):
        """Get appropriate collection based on memory type and NPC"""
        if npc and npc != "global":
            collection_name = f"npc_{npc}"
            if collection_name not in self.collections:
                self.collections[collection_name] = self._get_or_create_collection(collection_name)
            return self.collections[collection_name]
    
    # Default collections
        collection_map = {
            "player_action": "player_memories",
            "npc_interaction": "npc_memories", 
            "quest": "quests",
            "event": "global_events"
        }
        return self.collections[collection_map.get(memory_type, "global_events")]

    def _get_relevant_collections(self, npc_filter: str = None):
        """Get collections relevant to current context"""
        collections = [self.collections["global_events"]]
    
        if npc_filter:
            npc_collection = f"npc_{npc_filter}"
            if npc_collection in self.collections:
                collections.append(self.collections[npc_collection])
    
        collections.append(self.collections["player_memories"])
        return collections
        
    
    def _build_context(self, player_id: str, player_input: str) -> Dict:
        player = self.players[player_id]
    
    # NEW ENHANCED MEMORY SEARCH - REPLACE WITH THIS
        enhanced_context = self.build_enhanced_context(
            player_input=player_input,
            current_npc=self.current_npc,  # You'll need to track current NPC
            current_location=self.shared_world_state.get("location", "unknown")
        )
    
    # Build much richer memory context
        memory_context = ""
    
    # High relevance memories (score > 0.7)
        if enhanced_context["high_relevance_memories"]:
            memory_context = "🔴 CRITICAL MEMORIES:\n- " + "\n- ".join(enhanced_context["high_relevance_memories"])
    
    # Medium relevance memories (score 0.4-0.7)  
        if enhanced_context["medium_relevance_memories"]:
            memory_context += "\n\n🟡 RELATED MEMORIES:\n- " + "\n- ".join(enhanced_context["medium_relevance_memories"])
    
    # Add player-specific memories
        player_memories = self.search_memories(
            player['character'], 
            npc_filter=None,  # Get all memories about this player
            n_results=3
        )
        player_memory_texts = [mem for mem, score in player_memories if score > 0.5]
    
        if player_memory_texts:
            memory_context += f"\n\n👤 ABOUT {player['character'].upper()}:\n- " + "\n- ".join(player_memory_texts)
    
    # Debug info (optional)
        memory_context += f"\n\n📊 Memory search: {enhanced_context['total_memories_considered']} memories considered"
    def build_enhanced_context(self, player_input: str, current_npc: str = None, 
                          current_location: str = None) -> Dict:
        """Build context using enhanced memory retrieval"""
    
    # Get relevant memories with scoring
        relevant_memories = self.search_memories(
            query=player_input,
            npc_filter=current_npc,
            location_filter=current_location,
            n_results=5
        )
    
    # Separate high and medium relevance memories
        high_relevance = [mem for mem, score in relevant_memories if score > 0.7]
        medium_relevance = [mem for mem, score in relevant_memories if score > 0.4 and score <= 0.7]
    
        context = {
            "high_relevance_memories": high_relevance,
            "medium_relevance_memories": medium_relevance,
            "current_npc": current_npc,
            "current_location": current_location,
            "total_memories_considered": len(relevant_memories)
        }
    
        return context
    def _update_working_memory(self, player_id: str, player_input: str, dm_response: str):
        """Update working memory systems"""
        player = self.players[player_id]
        
        # Update player working memory
        player_turn = f"{player['character']}: {player_input}"
        if player_id in self.player_working_memory:
            self.player_working_memory[player_id].append(player_turn)
            if len(self.player_working_memory[player_id]) > 5:
                self.player_working_memory[player_id].pop(0)
        
        # Update global working memory
        global_turn = f"Turn {self.turn_count}: {player['character']} - {player_input[:50]}..."
        self.global_working_memory.append(global_turn)
        if len(self.global_working_memory) > 10:
            self.global_working_memory.pop(0)


In [161]:

def multi_agent_demo():
    """Demo the multi-agent system"""
    dm = EnhancedMultiplayerDungeonMaster()
    
    print("🚀 MULTI-AGENT DUNGEON MASTER DEMO")
    print("=" * 60)
    
    # Add a test player
    player_id = dm.add_player("TestPlayer", "Aragorn")
    
    test_scenarios = [
        "I draw my sword and enter the dark forest",
        "I listen carefully for any sounds in the trees",
        "I remember seeing a glowing stone earlier - where was it?",
        "I attack the goblin with my sword",
        "I search the goblin's body for treasure"
    ]
    
    for i, action in enumerate(test_scenarios, 1):
        print(f"\n{'='*50}")
        print(f"TURN {i}: {action}")
        print(f"{'='*50}")
        
        response, agent_log = dm.generate_dm_response(player_id, action)
        
        print(f"\n🎭 Aragorn: {action}")
        print(f"🤖 DM: {response}")
        
        # Show agent workings
        print(f"\n🔧 AGENT WORKINGS:")
        print(f"   Memories extracted: {len(agent_log['memories_extracted'])}")
        for memory in agent_log['memories_extracted']:
            print(f"     - {memory}")
        
        if agent_log['consistency_notes']:
            print(f"   Consistency notes: {agent_log['consistency_notes']}")
    
    # Show final memory state
    print(f"\n📊 FINAL MEMORY STATE:")
    memories = dm.search_memories("forest", "all", 5)
    for i, memory in enumerate(memories, 1):
        print(f"   {i}. {memory}")

def interactive_multi_agent_game():
    """Interactive game with multi-agent system"""
    dm = EnhancedMultiplayerDungeonMaster()
    
    print("🎮 INTERACTIVE MULTI-AGENT DUNGEON MASTER")
    print("=" * 60)
    print("Commands:")
    print("  join <name> <character> - Join the game")
    print("  play <name> <action> - Take an action")
    print("  memories <query> - Search memories")
    print("  status - Game status")
    print("  quit - Exit")
    print("=" * 60)
    
    player_id_map = {}
    
    while True:
        try:
            command = input("\nCommand: ").strip().split(' ', 2)
            
            if not command or not command[0]:
                continue
            
            cmd = command[0].lower()
            
            if cmd == 'quit':
                break
            
            elif cmd == 'join':
                if len(command) < 2:
                    print("Usage: join <name> [character]")
                    continue
                name = command[1]
                character = command[2] if len(command) > 2 else name
                player_id = dm.add_player(name, character)
                player_id_map[name.lower()] = player_id
                print(f"✅ Joined as {character}")
            
            elif cmd == 'play':
                if len(command) < 3:
                    print("Usage: play <name> <action>")
                    continue
                name = command[1]
                action = command[2]
                
                player_id = player_id_map.get(name.lower())
                if not player_id:
                    print("❌ Player not found. Join first.")
                    continue
                
                response, agent_log = dm.generate_dm_response(player_id, action)
                player_char = dm.players[player_id]['character']
                
                print(f"\n🎭 {player_char}: {action}")
                print(f"🤖 DM: {response}")
                
                # Show agent insights
                if agent_log['memories_extracted']:
                    print(f"\n💡 Memories stored: {len(agent_log['memories_extracted'])}")
            
            elif cmd == 'memories':
                if len(command) < 2:
                    print("Usage: memories <query>")
                    continue
                query = command[1]
                memories = dm.search_memories(query)
                if memories:
                    print(f"\n🔍 Memories for '{query}':")
                    for i, mem in enumerate(memories, 1):
                        print(f"  {i}. {mem}")
                else:
                    print("No memories found.")
            
            elif cmd == 'status':
                print(f"\n📊 Game Status:")
                print(f"  Turns: {dm.turn_count}")
                print(f"  Active Players: {len(dm.active_players)}")
                for pid in dm.active_players:
                    player = dm.players[pid]
                    print(f"    - {player['character']} ({player['name']})")
        
        except KeyboardInterrupt:
            print("\n\n👋 Thanks for playing!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")

In [162]:
def extended_multi_agent_demo():
    """50-turn comprehensive demo testing all features"""
    dm = EnhancedMultiplayerDungeonMaster()
    
    print("🚀 EXTENDED 50-TURN MULTI-AGENT DUNGEON MASTER DEMO")
    print("=" * 60)
    print("Testing: Dynamic Temperature • Enhanced Memory • Multi-Agent Pipeline")
    print("=" * 60)
    
    # Add players
    player1_id = dm.add_player("Alice", "Aragorn")
    player2_id = dm.add_player("Bob", "Gandalf")
    
    print(f"\n🎭 Party: Aragorn (Warrior) & Gandalf (Wizard)")
    print("📍 Starting at: Crossroads Inn")
    print("🌧️  Weather: Stormy evening")
    print("=" * 60)
    
    # 50 diverse test scenarios covering all emotion modes
    test_scenarios = [
        # Turns 1-10: Exploration & Lore (Low Temperature)
        (player1_id, "I look around the inn carefully"),
        (player2_id, "I study the ancient maps on the wall"),
        (player1_id, "I listen to the conversations in the tavern"),
        (player2_id, "I remember hearing about a prophecy long ago"),
        (player1_id, "I examine the strange symbols on the floor"),
        (player2_id, "What do I know about the Shadow King?"),
        (player1_id, "I search the room for hidden passages"),
        (player2_id, "I recall the legend of the Starstone"),
        (player1_id, "I observe the behavior of the other patrons"),
        (player2_id, "Who was the previous owner of this inn?"),
        
        # Turns 11-20: Story & Dialogue (Medium Temperature)
        (player1_id, "I approach the barkeep and ask for information"),
        (player2_id, "I talk to the old man in the corner about the storm"),
        (player1_id, "I buy a round of drinks for the house"),
        (player2_id, "I share stories of my travels with the children"),
        (player1_id, "I ask about work for a skilled swordsman"),
        (player2_id, "I inquire about magical disturbances in the area"),
        (player1_id, "I challenge a patron to a friendly arm wrestling match"),
        (player2_id, "I offer to read fortunes for the curious"),
        (player1_id, "I share my dream about a dark forest"),
        (player2_id, "I discuss the politics of the nearby kingdoms"),
        
        # Turns 21-30: Emotional & Reflective (Medium-High Temperature)
        (player1_id, "I feel anxious about the coming journey"),
        (player2_id, "I remember my fallen mentor with sadness"),
        (player1_id, "I confess my fears about the darkness ahead"),
        (player2_id, "I feel the weight of my magical responsibilities"),
        (player1_id, "I look at my sword and remember past battles"),
        (player2_id, "I ponder the meaning of the ancient prophecy"),
        (player1_id, "I feel a strange connection to this place"),
        (player2_id, "I sense great magic sleeping beneath us"),
        (player1_id, "I miss my homeland in the northern mountains"),
        (player2_id, "I worry about the balance of power shifting"),
        
        # Turns 31-40: Battle & Action (High Temperature)
        (player1_id, "I draw my sword as bandits burst through the door!"),
        (player2_id, "I cast a protective shield around the innocent!"),
        (player1_id, "I attack the lead bandit with all my strength!"),
        (player2_id, "I unleash a lightning bolt at the attackers!"),
        (player1_id, "I scream a battle cry and charge forward!"),
        (player2_id, "I summon magical winds to knock them down!"),
        (player1_id, "I defend the children with my shield!"),
        (player2_id, "I create an illusion of reinforcements!"),
        (player1_id, "I strike down the last bandit standing!"),
        (player2_id, "I heal the wounded with my magic!"),
        
        # Turns 41-50: Mixed & Complex Scenarios
        (player1_id, "I search the bandits for clues about their leader"),
        (player2_id, "I remember seeing their symbol somewhere before"),
        (player1_id, "I feel exhausted but victorious after the fight"),
        (player2_id, "I detect dark magic residue on the bandits"),
        (player1_id, "I suggest we follow their tracks at dawn"),
        (player2_id, "I warn that darker forces may be behind this"),
        (player1_id, "I look at Gandalf with newfound respect"),
        (player2_id, "I sense our fates are now intertwined"),
        (player1_id, "I prepare my gear for the journey ahead"),
        (player2_id, "I cast a final protective ward around the inn")
    ]
    
    # Track statistics
    stats = {
        "lore_check": 0,
        "exploration": 0,
        "emotional": 0,
        "story": 0,
        "battle": 0,
        "memories_stored": 0,
        "consistency_checks": 0
    }
    
    print("\n🎬 BEGINNING 50-TURN ADVENTURE...\n")
    
    for turn, (player_id, action) in enumerate(test_scenarios, 1):
        player_name = dm.players[player_id]['character']
        
        print(f"\n{'='*50}")
        print(f"🎯 TURN {turn}/50 - {player_name}")
        print(f"{'='*50}")
        print(f"🗣️  {player_name}: {action}")
        
        # Generate response
        response, agent_log = dm.generate_dm_response(player_id, action)
        
        print(f"🤖 DM: {response}")
        
        # Update statistics
        context_type = dm.detect_context_type(action)
        stats[context_type] += 1
        stats["memories_stored"] += len(agent_log['memories_extracted'])
        if agent_log['consistency_notes']:
            stats["consistency_checks"] += 1
        
        # Show agent workings every 10 turns
        if turn % 10 == 0:
            print(f"\n🔧 TURN {turn} - AGENT WORKINGS:")
            print(f"   Memories extracted: {len(agent_log['memories_extracted'])}")
            for memory in agent_log['memories_extracted'][:2]:  # Show first 2
                print(f"     - {memory}")
            if agent_log['consistency_notes']:
                print(f"   Consistency notes: {agent_log['consistency_notes'][0][:100]}...")
            
            # Memory status every 10 turns
            print(f"\n📊 MEMORY STATUS (Turn {turn}):")
            recent_memories = dm.search_memories("recent", n_results=3)
            print(f"   Recent memories: {len(recent_memories)}")
            for i, (mem, score) in enumerate(recent_memories, 1):
                print(f"     {i}. [{score:.2f}] {mem[:60]}...")
    
    # Final comprehensive report
    print(f"\n{'='*60}")
    print("📈 50-TURN DEMO COMPLETE - FINAL REPORT")
    print(f"{'='*60}")
    
    print(f"\n🎭 EMOTION MODE DISTRIBUTION:")
    total_turns = sum(stats.values() for key in ['lore_check', 'exploration', 'emotional', 'story', 'battle'])
    for mode, count in [(k, v) for k, v in stats.items() if k in ['lore_check', 'exploration', 'emotional', 'story', 'battle']]:
        percentage = (count / total_turns) * 100
        print(f"   {mode.upper():<12}: {count:2d} turns ({percentage:5.1f}%)")
    
    print(f"\n💾 MEMORY SYSTEM PERFORMANCE:")
    print(f"   Total memories stored: {stats['memories_stored']}")
    print(f"   Consistency checks: {stats['consistency_checks']}")
    print(f"   Total game turns: {dm.turn_count}")
    
    print(f"\n🔍 FINAL MEMORY ANALYSIS:")
    
    # Test different types of memory recall
    test_queries = [
        ("bandits", "Recent combat"),
        ("prophecy", "Lore knowledge"), 
        ("Gandalf", "Character memories"),
        ("inn", "Location details")
    ]
    
    for query, description in test_queries:
        memories = dm.search_memories(query, n_results=2)
        print(f"\n   {description} ('{query}'):")
        if memories:
            for i, (mem, score) in enumerate(memories, 1):
                print(f"     {i}. [{score:.2f}] {mem}")
        else:
            print("     No relevant memories found")
    
    print(f"\n🎯 KEY ACHIEVEMENTS:")
    achievements = []
    if stats['battle'] >= 5:
        achievements.append("✅ Dynamic battle scenes with high temperature")
    if stats['lore_check'] >= 5:
        achievements.append("✅ Consistent lore recall with low temperature")
    if stats['memories_stored'] >= 15:
        achievements.append("✅ Effective memory extraction and storage")
    if any(score > 0.8 for _, score in dm.search_memories("", n_results=10)):
        achievements.append("✅ High-relevance memory retrieval")
    
    for achievement in achievements:
        print(f"   {achievement}")
    
    print(f"\n🎉 DEMO COMPLETED SUCCESSFULLY!")
    print("The AI Dungeon Master has demonstrated:")
    print("  • Emotional intelligence through dynamic temperature")
    print("  • Long-term consistency through multi-agent pipeline")
    print("  • Context-aware memory through enhanced retrieval")
    print(f"{'='*60}")



In [163]:
if __name__ == "__main__":
    print("Choose demo mode:")
    print("1. Interactive Multi-Agent Game")
    print("2. Quick 7-Turn Demo") 
    print("3. Extended 50-Turn Comprehensive Demo")
    
    choice = input("\nEnter 1, 2 or 3: ").strip()
    
    if choice == "1":
        interactive_multi_agent_game()
    elif choice == "2":
        multi_agent_demo()
    else:
        extended_multi_agent_demo()

Choose demo mode:
1. Interactive Multi-Agent Game
2. Quick 7-Turn Demo
3. Extended 50-Turn Comprehensive Demo



Enter 1, 2 or 3:  2


🎮 Multi-Agent Dungeon Master initialized! Session ID: 338d6ce8
🚀 MULTI-AGENT DUNGEON MASTER DEMO


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

💾 [PLAYER_JOIN] Player TestPlayer joined the game as Aragorn (importance: 0.7)
🎮 Player TestPlayer joined as Aragorn (ID: fc17cf92)

TURN 1: I draw my sword and enter the dark forest

🔄 Turn 1 - Multi-agent processing for Aragorn...
🎭 Emotion Mode: story | Temperature: 0.7


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

💾 [EVENT] The poem hints at the location of the Starlight Crystal, mentioning the whispering woods and ancient secrets. (importance: 1.0)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

💾 [EVENT] The party found a glowing stone, but its location was not specified. (importance: 1.0)
   📚 Lore Keeper: Checking consistency...

🎭 Aragorn: I search the goblin's body for treasure
🤖 DM: 1. Check for contradictions with established facts
2. Identify missing continuity elements
3. Suggest minor adjustments to maintain consistency

**ISSUES:**

- The current action is described as Aragorn searching the goblin's body for treasure, but the context suggests Eira is the half-elf rogue performing the action.
- The party context mentions Eira, Morgran, and Lyra, but Aragorn is not mentioned as a party member.
- The recent conversation and current action are described from Eira's perspective, but the initial action was Aragorn's.

**ADJUSTMENTS:**

To maintain consistency, the current action should be adjusted to reflect Eira's perspective. The party context should be revised to include Aragorn as a party member.

🔧 AGENT WORKINGS:
   Memories extracted: 8
     - Here are the extracte

Batches:   0%|          | 0/1 [00:00<?, ?it/s]